## 1. 엑셀에서 데이터 추출

In [10]:
import pandas as pd
import re
import os

# 1. DataFrame에서 특정 패턴과 일치하는 행을 찾는 함수
def find_row_by_pattern(df, pattern):
    for idx, row in df.iterrows():
        if any(re.search(pattern, str(cell), re.IGNORECASE) for cell in row):
            return row
    return None

# 2. 엑셀 파일에서 특정 항목에 해당하는 값을 추출하는 함수
def extract_financial_data(file_path, sheet_names, data_items, date_columns):
    results = []
    try:
        xls = pd.ExcelFile(file_path)
        available_sheets = xls.sheet_names

        for sheet_name in sheet_names:
            if sheet_name in available_sheets:
                df = pd.read_excel(file_path, sheet_name=sheet_name, header=[0, 1])
                df.columns = [f'{col[0]} {col[1]}' if 'Unnamed' not in col[1] else col[0] for col in df.columns]

                # 시트의 표기 금액이 천단위일때
                unit_in_thousands = any('in Thousands' in col for col in df.columns)
                # 시트의 표기 금액이 만단위일때
                unit_in_millions = any('in Millions' in col for col in df.columns)

                # 각 항목에 대해 패턴을 기반으로 데이터를 추출
                items = data_items.get(sheet_name, {})

                for item, pattern in items.items():
                    row = find_row_by_pattern(df, pattern)
                    if row is not None:
                        for date in date_columns:
                            matching_col = [col for col in df.columns if re.search(date, col, re.IGNORECASE)]
                            if matching_col:
                                col_name = matching_col[0]
                                value = row.get(col_name, None)
                                if value is not None:
                                    try:
                                        # 괄호 처리를 통해 음수 값 변환 및 수치 변환
                                        if isinstance(value, str) and '(' in value and ')' in value:
                                            value = -float(value.replace('(', '').replace(')', '').replace(',', ''))
                                        elif isinstance(value, str):
                                            value = float(value.replace(',', ''))
                                        # 단위 조정
                                        if unit_in_thousands:
                                            value = round(value / 1000, 1)  # Thousands to Millions 변환
                                        elif unit_in_millions:
                                            value = round(value, 1)  # Millions 단위 그대로 유지
                                        else:
                                            value = round(value / 1000000, 1)  # Dollars to Millions 변환
                                        results.append({
                                            'Date': date,
                                            'Item': item,
                                            'Value': value
                                        })
                                    except ValueError:
                                        print(f"Skipping non-numeric data in {sheet_name} at {date} for {item}")
                            else:
                                results.append({
                                    'Date': date,
                                    'Item': item,
                                    'Value': None
                                })
                    else:
                        for date in date_columns:
                            results.append({
                                'Date': date,
                                'Item': item,
                                'Value': None
                            })
        return results

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return []

# 3. 분기 데이터를 조정하는 함수
def adjust_quarterly_data(df):
    for item in df['Item'].unique():
        item_data = df[df['Item'] == item]
        for year in range(2018, 2024):
            q1 = f'Mar. 31, {year}'
            q2 = f'Jun. 30, {year}'
            q3 = f'Sep. 30, {year}'
            q4 = f'Dec. 31, {year}'
            mask = item_data['Date'].isin([q1, q2, q3, q4])
            if mask.sum() == 4:
                q1_value = item_data[mask & (item_data['Date'] == q1)]['Value'].values[0]
                q2_value = item_data[mask & (item_data['Date'] == q2)]['Value'].values[0]
                q3_value = item_data[mask & (item_data['Date'] == q3)]['Value'].values[0]
                q4_value = item_data[mask & (item_data['Date'] == q4)]['Value'].values[0]
                adjusted_q4_value = q4_value - q3_value - q2_value - q1_value
                df.loc[mask & (df['Date'] == q4), 'Value'] = adjusted_q4_value
    return df

# 기준금리와 주가 데이터를 추가 데이터로 정의
additional_data = {
    'base_rate': {
        'Mar. 31, 2018': 1.625, 'Jun. 30, 2018': 1.875, 'Sep. 30, 2018': 2.125, 'Dec. 31, 2018': 2.375,
        'Mar. 31, 2019': 2.375, 'Jun. 30, 2019': 2.375, 'Sep. 30, 2019': 1.875, 'Dec. 31, 2019': 1.625,
        'Mar. 31, 2020': 0.125, 'Jun. 30, 2020': 0.125, 'Sep. 30, 2020': 0.125, 'Dec. 31, 2020': 0.125,
        'Mar. 31, 2021': 0.125, 'Jun. 30, 2021': 0.125, 'Sep. 30, 2021': 0.125, 'Dec. 31, 2021': 0.125,
        'Mar. 31, 2022': 0.375, 'Jun. 30, 2022': 1.625, 'Sep. 30, 2022': 3.125, 'Dec. 31, 2022': 4.375,
        'Mar. 31, 2023': 4.875, 'Jun. 30, 2023': 5.125, 'Sep. 30, 2023': 5.375, 'Dec. 31, 2023': 5.375,
        'Mar. 31, 2024': 5.375
    }
}

# 파일 경로 및 시트 이름 설정
directory = '/content/'  # 실제 디렉토리 경로로 변경해주세요
sheet_names = ['CONSOLIDATED STATEMENTS OF COND', 'CONSOLIDATED STATEMENTS OF INCO', 'Consolidated Statements Of Cond', 'Consolidated Statements Of Inco', 'Consolidated Statements of Cond', 'Consolidated Statements of Inco']

# 데이터 항목 및 패턴 정의
income_data_items = {
    'Interest Income': r'Total interest income',
    'Net Interest Income': r'Net interest income',
    'Non-Interest Income': r'Total non-interest income',
    'Income Before Tax': r'Income before taxes',
    'Tax Expense': r'Income tax expense',
    'Net Income': r'Net income'
}

balance_sheet_items = {
    'Cash and Due from Banks': r'Cash and due from banks',
    'Goodwill': r'Goodwill',
    'Loans': r'Total loans',
    'Net Loans': r'Loans\, net of unearned income\, excluding covered loans|Loans\, net of unearned income',
    'Total Assets': r'Total assets',
    'Total Liabilities': r'Total liabilities',
    'Total Stockholders Equity': r"Total shareholders’ equity",
    'Retained Earnings': r'Retained earnings'
}

data_items = {
    'Consolidated Statements Of Cond': balance_sheet_items,
    'Consolidated Statements Of Inco': income_data_items,
    'Consolidated Statements of Inco': income_data_items,
    'Consolidated Statements of Cond': balance_sheet_items,
    'CONSOLIDATED STATEMENTS OF INCO': income_data_items,
    'CONSOLIDATED STATEMENTS OF COND': balance_sheet_items

}

# 날짜 열 정의
date_columns = [
    'Mar. 31, 2018', 'Jun. 30, 2018', 'Sep. 30, 2018', 'Dec. 31, 2018',
    'Mar. 31, 2019', 'Jun. 30, 2019', 'Sep. 30, 2019', 'Dec. 31, 2019',
    'Mar. 31, 2020', 'Jun. 30, 2020', 'Sep. 30, 2020', 'Dec. 31, 2020',
    'Mar. 31, 2021', 'Jun. 30, 2021', 'Sep. 30, 2021', 'Dec. 31, 2021',
    'Mar. 31, 2022', 'Jun. 30, 2022', 'Sep. 30, 2022', 'Dec. 31, 2022',
    'Mar. 31, 2023', 'Jun. 30, 2023', 'Sep. 30, 2023', 'Dec. 31, 2023',
    'Mar. 31, 2024'
]

# 결과를 저장할 리스트 초기화
all_results = []

# 디렉토리 내의 모든 엑셀 파일 처리
for filename in os.listdir(directory):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        bank_id = filename.split('.')[0]
        file_path = os.path
        file_path = os.path.join(directory, filename)
        financial_data = extract_financial_data(file_path, sheet_names, data_items, date_columns)

        if financial_data:
            all_results.extend(financial_data)

# 데이터프레임으로 변환
result_df = pd.DataFrame(all_results)

# Date를 Year와 Quarter로 분리
result_df['Year'] = result_df['Date'].apply(lambda x: x.split(', ')[1])
result_df['Quarter'] = result_df['Date'].apply(lambda x: x.split(', ')[0])

# 피벗 테이블로 재구성
result_pivot = result_df.pivot_table(index=['Year', 'Quarter'], columns='Item', values='Value', aggfunc='first').reset_index()

# 기준금리 추가
base_rate_df = pd.DataFrame(additional_data['base_rate'].items(), columns=['Date', 'Interest_Rate'])
base_rate_df['Year'] = base_rate_df['Date'].apply(lambda x: x.split(', ')[1])
base_rate_df['Quarter'] = base_rate_df['Date'].apply(lambda x: x.split(', ')[0])
base_rate_df.drop(columns='Date', inplace=True)

# 최종 데이터프레임과 병합
final_df = pd.merge(result_pivot, base_rate_df, on=['Year', 'Quarter'], how='left')

# 작업 시 마다 티커명을 바꿔야합니다.
# 기존 데이터프레임에 'Name' 열을 추가하고 이를 맨 앞에 배치
final_df['Name'] = 'WTFC'

# 열 순서를 조정하여 'Name' 열을 맨 앞으로 이동
cols = ['Name'] + [col for col in final_df.columns if col != 'Name']
final_df = final_df[cols]

# 결과 출력
final_df


,Name,Year,Quarter,Cash and Due from Banks,Goodwill,Income Before Tax,Interest Income,Loans,Net Income,Net Interest Income,Net Loans,Non-Interest Income,Retained Earnings,Tax Expense,Total Assets,Total Liabilities,Total Stockholders Equity,Interest_Rate
0,WTFC,2018,Dec. 31,392.1,573.1,460.1,1170.8,NaN,343.2,964.9,23820.7,356.1,1610.6,117.0,31244.8,27977.3,3267.6,2.375
1,WTFC,2018,Jun. 30,304.6,510.0,121.6,284.0,22610.6,89.6,238.2,22610.6,95.2,1464.5,32.0,29464.6,26357.7,3106.9,1.875
2,WTFC,2018,Mar. 31,231.4,511.5,108.1,261.2,22062.1,82.0,225.1,22062.1,85.7,1387.7,26.1,28456.8,25425.5,3031.2,1.625
3,WTFC,2018,Sep. 30,279.9,537.6,122.8,305.0,23124.0,91.9,247.6,23124.0,99.9,1543.7,30.9,30142.7,26962.9,3179.8,2.125
4,WTFC,2019,Dec. 31,286.2,645.2,480.1,1385.1,NaN,355.7,1054.9,26800.3,407.2,1899.6,124.4,36620.6,32929.3,3691.2,1.625
5,WTFC,2019,Jun. 30,300.9,584.9,110.2,346.8,NaN,81.5,266.2,25304.7,98.2,1747.3,28.7,33641.8,30194.8,3446.9,2.375
6,WTFC,2019,Mar. 31,270.8,573.7,118.6,334.0,NaN,89.1,262.0,24214.6,81.7,1682.0,29.5,32358.6,28986.6,3372.0,2.375
7,WTFC,2019,Sep. 30,448.8,584.3,134.6,354.6,NaN,99.1,264.9,25710.2,115.1,1830.2,35.5,34911.9,31371.6,3540.3,1.875
8,WTFC,2020,Dec. 31,322.4,645.7,389.8,1293.0,NaN,293.0,1039.9,32079.1,604.2,2080.0,96.8,45080.8,40964.8,4116.0,0.125
9,WTFC,2020,Jun. 30,345.0,644.2,30.7,329.8,NaN,21.7,263.1,31402.9,162.0,1921.0,9.0,43540.0,39549.8,3990.2,0.125


## 2. 주가 데이터 로드

In [11]:
import pandas as pd

# 주가가 담긴 CSV 파일 읽기
df_stock = pd.read_csv('stock.csv')

# 'Date' 열을 'Year'와 'Quarter'로 분리
df_stock['Year'] = df_stock['Date'].apply(lambda x: x.split('/')[0])
df_stock['Quarter'] = df_stock['Date'].apply(lambda x: x.split('/')[1])
df_stock['Quarter'] = df_stock['Quarter'].apply(lambda x: f'Mar. 31' if x == '03' else (f'Jun. 30' if x == '06' else (f'Sep. 30' if x == '09' else f'Dec. 31')))

# 열 순서 변경
df_stock = df_stock[['Year', 'Quarter'] + [col for col in df_stock.columns if col not in ['Year', 'Quarter']]]

# 결과 출력
df_stock.head()


,Year,Quarter,Date,base_rate,BANF,BOKF,CADE,COF,C,CMA,...,WAFD,AX,CUBI,AUB,WSFS,HTLF,INDB,HOPE,IBTX,PPBI
0,2018,Mar. 31,2018/03,1.625,46.171,84.129,24.976,85.525,54.265,71.852,...,28.691,40.53,29.15,29.996,44.307,46.025,59.484,13.635,61.221,31.984
1,2018,Jun. 30,2018/06,1.875,51.681,80.246,25.987,82.382,54.053,68.305,...,27.259,40.91,28.38,31.935,49.407,47.701,65.521,13.465,57.957,30.353
2,2018,Sep. 30,2018/09,2.125,52.520,83.459,25.896,85.455,58.312,68.006,...,26.818,34.39,23.53,31.826,43.801,50.602,69.363,12.313,57.640,29.597
3,2018,Dec. 31,2018/12,2.375,43.934,63.263,20.804,68.348,42.607,52.118,...,22.528,25.18,18.20,23.473,35.308,38.411,59.315,9.125,39.886,20.304
4,2019,Mar. 31,2019/03,2.375,46.193,70.763,22.592,74.239,51.280,56.095,...,24.531,28.96,18.31,27.075,36.043,37.441,68.718,10.161,44.900,21.261


## 3. 주가데이터 병합

In [12]:
import pandas as pd

name = final_df['Name'][0]

print(name)
# Year 열을 문자열로 변환
final_df['Year'] = final_df['Year'].astype(str)

# stock.csv 파일 읽기
df_stock = pd.read_csv('stock.csv')

# 'Date' 열을 'Year'와 'Quarter'로 분리
df_stock['Year'] = df_stock['Date'].apply(lambda x: x.split('/')[0])
df_stock['Quarter'] = df_stock['Date'].apply(lambda x: x.split('/')[1])
df_stock['Quarter'] = df_stock['Quarter'].apply(lambda x: f'Mar. 31' if x == '03' else (f'Jun. 30' if x == '06' else (f'Sep. 30' if x == '09' else f'Dec. 31')))

# Year 열을 문자열로 변환
df_stock['Year'] = df_stock['Year'].astype(str)

# BANF 주가 데이터만 추출
df_banf_stock = df_stock[['Year', 'Quarter', name]]

# test.csv와 병합
df_merged = pd.merge(final_df, df_banf_stock, on=['Year', 'Quarter'], how='left')

# 열 이름을 'Stock_Price'로 변경
df_merged.rename(columns={name: 'Stock_Price'}, inplace=True)

# 병합된 데이터프레임을 test_merged.csv로 저장
df_merged.to_csv(f'{name}_merged.csv', index=False)

# 결과 출력
df_merged.head()


WTFC


,Name,Year,Quarter,Cash and Due from Banks,Goodwill,Income Before Tax,Interest Income,Loans,Net Income,Net Interest Income,Net Loans,Non-Interest Income,Retained Earnings,Tax Expense,Total Assets,Total Liabilities,Total Stockholders Equity,Interest_Rate,Stock_Price
0,WTFC,2018,Dec. 31,392.1,573.1,460.1,1170.8,NaN,343.2,964.9,23820.7,356.1,1610.6,117.0,31244.8,27977.3,3267.6,2.375,60.239
1,WTFC,2018,Jun. 30,304.6,510.0,121.6,284.0,22610.6,89.6,238.2,22610.6,95.2,1464.5,32.0,29464.6,26357.7,3106.9,1.875,78.506
2,WTFC,2018,Mar. 31,231.4,511.5,108.1,261.2,22062.1,82.0,225.1,22062.1,85.7,1387.7,26.1,28456.8,25425.5,3031.2,1.625,77.445
3,WTFC,2018,Sep. 30,279.9,537.6,122.8,305.0,23124.0,91.9,247.6,23124.0,99.9,1543.7,30.9,30142.7,26962.9,3179.8,2.125,76.767
4,WTFC,2019,Dec. 31,286.2,645.2,480.1,1385.1,NaN,355.7,1054.9,26800.3,407.2,1899.6,124.4,36620.6,32929.3,3691.2,1.625,65.152


In [13]:
import pandas as pd

# 파일 경로 지정
file_path = f'/content/{name}_merged.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 데이터프레임 내용 확인
print(df.head())

# 데이터 정렬: 1. 회사 이름(Name)별, 2. 년도(Year)별, 3. 분기(Quarter)별
quarter_order = {'Mar. 31': 1, 'Jun. 30': 2, 'Sep. 30': 3, 'Dec. 31': 4}
df['Quarter_Order'] = df['Quarter'].map(quarter_order)
df_sorted = df.sort_values(by=['Name', 'Year', 'Quarter_Order']).drop(columns=['Quarter_Order'])

# 정렬된 데이터프레임 저장
sorted_file_path = f'/content/sorted_{name}_scaled.csv'
df_sorted.to_csv(sorted_file_path, index=False)

# 정렬된 CSV 파일 다운로드 링크 제공
sorted_file_path


   Name  Year  Quarter  Cash and Due from Banks  Goodwill  Income Before Tax  \
0  WTFC  2018  Dec. 31                    392.1     573.1              460.1   
1  WTFC  2018  Jun. 30                    304.6     510.0              121.6   
2  WTFC  2018  Mar. 31                    231.4     511.5              108.1   
3  WTFC  2018  Sep. 30                    279.9     537.6              122.8   
4  WTFC  2019  Dec. 31                    286.2     645.2              480.1   

   Interest Income    Loans  Net Income  Net Interest Income  Net Loans  \
0           1170.8      NaN       343.2                964.9    23820.7   
1            284.0  22610.6        89.6                238.2    22610.6   
2            261.2  22062.1        82.0                225.1    22062.1   
3            305.0  23124.0        91.9                247.6    23124.0   
4           1385.1      NaN       355.7               1054.9    26800.3   

   Non-Interest Income  Retained Earnings  Tax Expense  Total Assets

'/content/sorted_WTFC_scaled.csv'

## 4. 10-K 데이터를 분기 데이터로 변환

In [14]:
import pandas as pd

def adjust_quarterly_data(input_file_path, output_file_path):
    # Load the CSV file
    df = pd.read_csv(input_file_path)

    # Identify the columns that need quarterly adjustments
    columns_to_adjust = ['Income Before Tax', 'Interest Income', 'Net Income',
                         'Net Interest Income', 'Non-Interest Income', 'Tax Expense']

    # Create a copy of the dataframe to avoid modifying the original data
    df_adjusted = df.copy()

    # Perform the quarterly adjustments
    for year in df_adjusted['Year'].unique():
        for col in columns_to_adjust:
            # Filter the data for the specific year
            yearly_data = df_adjusted[df_adjusted['Year'] == year]

            # Get the values for Q1, Q2, Q3, and Q4
            q1_value = yearly_data[yearly_data['Quarter'] == 'Mar. 31'][col].values
            q2_value = yearly_data[yearly_data['Quarter'] == 'Jun. 30'][col].values
            q3_value = yearly_data[yearly_data['Quarter'] == 'Sep. 30'][col].values
            q4_value = yearly_data[yearly_data['Quarter'] == 'Dec. 31'][col].values

            # Ensure we have values for all quarters
            if len(q1_value) > 0 and len(q2_value) > 0 and len(q3_value) > 0 and len(q4_value) > 0:
                # Calculate the correct Q4 value
                corrected_q4_value = q4_value[0] - (q1_value[0] + q2_value[0] + q3_value[0])

                # Update the Q4 value in the dataframe and round to one decimal place
                df_adjusted.loc[(df_adjusted['Year'] == year) & (df_adjusted['Quarter'] == 'Dec. 31'), col] = round(corrected_q4_value, 1)

    # Save the adjusted dataframe to a new CSV file
    df_adjusted.to_csv(output_file_path, index=False)

# Example usage:
input_file_path = f'/content/{name}_merged.csv'
output_file_path = f'/content/{name}_scaled.csv'
adjust_quarterly_data(input_file_path, output_file_path)


## 5. 기존 데이터셋과 병합

In [15]:
import os
import pandas as pd

# List of file names
file_names = [
    'final_bank.csv', 'WTFC_scaled.csv'
]

# File directory (assuming they are in the same directory as the initial file)
file_directory = '/content/'

# Initialize an empty DataFrame
merged_data = pd.DataFrame()

# Read and concatenate each CSV file
for file_name in file_names:
    file_path = os.path.join(file_directory, file_name)
    df = pd.read_csv(file_path)
    merged_data = pd.concat([merged_data, df], ignore_index=True)


merged_data.to_csv('final.csv', index=False)
# Display the merged data
merged_data



,Name,Year,Quarter,Cash and Due from Banks,Goodwill,Income Before Tax,Interest Income,Loans,Net Income,Net Interest Income,Net Loans,Non-Interest Income,Retained Earnings,Tax Expense,Total Assets,Total Liabilities,Total Stockholders Equity,Interest_Rate,Stock_Price
0,BANF,2018,Mar. 31,181.9,79.8,36.9,70.9,4984.5,29.6,63.0,4932.9,30.1,661.3,7.3,7615.6,6777.5,838.1,1.625,46.171
1,BANF,2018,Jun. 30,188.5,79.7,39.8,75.1,5007.5,30.6,64.9,4955.3,30.4,684.4,9.2,7623.0,6761.0,862.0,1.875,51.681
2,BANF,2018,Sep. 30,185.0,79.7,41.9,77.4,4947.5,32.9,65.7,4895.7,32.8,707.5,9.0,7602.4,6717.6,884.8,2.125,52.520
3,BANF,2018,Dec. 31,228.4,79.7,41.2,79.8,4976.0,32.7,66.9,4924.6,31.9,722.6,8.4,7574.3,6671.5,902.8,2.375,43.934
4,BANF,2019,Mar. 31,186.0,79.7,41.0,80.9,5042.5,31.8,66.9,4989.6,32.0,744.7,9.2,7709.0,6781.1,927.9,2.375,46.193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,WTFC,2023,Mar. 31,445.9,653.6,243.6,639.7,39941.3,180.2,458.0,39565.5,107.8,2997.3,63.4,52873.5,47858.0,5015.5,4.875,71.108
396,WTFC,2023,Jun. 30,513.9,656.7,211.4,697.2,41410.8,154.8,447.5,41023.4,113.0,3120.6,56.7,54286.2,49244.3,5041.9,5.125,71.240
397,WTFC,2023,Sep. 30,418.1,656.1,224.9,762.4,41845.2,164.2,462.4,41446.0,112.5,3253.3,60.7,55555.2,50539.6,5015.6,5.375,74.420
398,WTFC,2023,Dec. 31,423.4,656.7,165.2,793.8,42559.1,123.4,470.0,42131.8,100.8,3345.4,41.7,56259.9,50860.4,5399.5,5.375,91.885
